In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.apache.spark::spark-mllib:2.3.0`
import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import $exclude.$                        , $ivy.$                            // for cleaner logs

import $profile.$           

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                    

import $ivy.$                                   

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

In [2]:
import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._

import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

In [7]:
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types.LongType
import org.apache.spark.sql.types.StringType

import org.apache.spark.sql.types.StructField

import org.apache.spark.sql.types.StructType

import org.apache.spark.sql.types.LongType

import org.apache.spark.sql.types.StringType

In [3]:
val sparkSession = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  // .yarn("/etc/hadoop/conf") // optional, for Spark on YARN - argument is the Hadoop conf directory
  // .emr("2.6.4") // on AWS ElasticMapReduce, this adds aws-related to the spark jar list
  .master("local") // change to "yarn-client" on YARN
  // .config("spark.executor.instances", "10")
  // .config("spark.executor.memory", "3g")
  // .config("spark.hadoop.fs.s3a.access.key", awsCredentials._1)
  // .config("spark.hadoop.fs.s3a.secret.key", awsCredentials._2)
  .appName("notebook")
  .getOrCreate()

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


sparkSession: SparkSession = org.apache.spark.sql.SparkSession@1a522a2a

## Train dataset

In [4]:
var train = sparkSession.read.parquet("/home/mahjoubi/Documents/github/toxic_comment/data/parquet/train")

val firstRow = Seq(train.first())
val data = sparkSession.sparkContext.parallelize(firstRow)
data.collect()

val schema = List(StructField("id", StringType, true),
StructField("comment_text", StringType, true),
StructField("toxic", LongType, true),
StructField("severe_toxic", LongType, true),
StructField("obscene", LongType, true),
StructField("threat", LongType, true),
StructField("insult", LongType, true),
StructField("identity_hate", LongType, true))

val dataSet = sparkSession.createDataFrame(data, StructType(schema))

dataSet.show()

dataSet.coalesce(1).write.parquet("/home/mahjoubi/Documents/github/toxic_comment/src/test/ressources/data/train")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


train: DataFrame = [id: string, comment_text: string ... 6 more fields]

## Test dataset

In [11]:
var test = sparkSession.read.parquet("/home/mahjoubi/Documents/github/toxic_comment/data/parquet/test")

val firstRow = Seq(test.first())
val data = sparkSession.sparkContext.parallelize(firstRow)
data.collect()

val schema = List(StructField("id", StringType, true),
                  StructField("comment_text", StringType, true))

val dataSet = sparkSession.createDataFrame(data, StructType(schema))

dataSet.show()

dataSet.coalesce(1).write.parquet("/home/mahjoubi/Documents/github/toxic_comment/src/test/resources/data/test")

+----------------+--------------------+
|              id|        comment_text|
+----------------+--------------------+
|5f66e39ac2caf365|== name == 

 Whi...|
+----------------+--------------------+



test: DataFrame = [id: string, comment_text: string]
firstRow: Seq[Row] = List(
  [5f66e39ac2caf365,== name == 

 While I accept that an organization can use improper spelling and grammar in their official name (e.g. Sheffield Wednesday Womens F.C., Magee-Womens Hospital), I am not sure this is the case here. The website uses both proper spelling (e.g. here) and improper. Unless an official source is brought that the improper spelling is official (an official logo also works) I'm going to use proper spelling.]
)
data: rdd.RDD[Row] = ParallelCollectionRDD[36] at parallelize at cmd10.sc:4
res10_3: Array[Row] = Array(
  [5f66e39ac2caf365,== name == 

 While I accept that an organization can use improper spelling and grammar in their official name (e.g. Sheffield Wednesday Womens F.C., Magee-Womens Hospital), I am not sure this is the case here. The website uses both proper spelling (e.g. here) and improper. Unless an official source is brought that the improper spelling is official (an of

In [10]:
dataSet.collect()

res9: Array[Row] = Array(
  [5f66e39ac2caf365,== name == 

 While I accept that an organization can use improper spelling and grammar in their official name (e.g. Sheffield Wednesday Womens F.C., Magee-Womens Hospital), I am not sure this is the case here. The website uses both proper spelling (e.g. here) and improper. Unless an official source is brought that the improper spelling is official (an official logo also works) I'm going to use proper spelling.]
)